## Deliverable 2. Create a Customer Travel Destinations Map.

In [95]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [96]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\emadn\Weather_Database\Weather_Database.csv")
city_data_df.head()

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Terekli-Mekteb,RU,44.1671,45.8698,66.34,42,48,5.53,scattered clouds
1,1,Port Elizabeth,ZA,-33.9180,25.5701,67.10,78,75,13.80,broken clouds
2,2,Ponta Delgada,PT,37.7333,-25.6667,61.52,67,40,28.77,scattered clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.84,79,20,21.27,light rain
4,4,Rikitea,PF,-23.1203,-134.9692,78.89,74,6,17.38,clear sky


In [97]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [98]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Atuona,PF,-9.8000,-139.0333,78.84,79,20,21.27,light rain
4,4,Rikitea,PF,-23.1203,-134.9692,78.89,74,6,17.38,clear sky
6,6,Dingle,PH,10.9995,122.6711,76.10,88,100,4.36,overcast clouds
7,7,Tabou,CI,4.4230,-7.3528,86.07,69,92,10.54,overcast clouds
9,9,Waingapu,ID,-9.6567,120.2641,78.82,83,96,3.51,overcast clouds
10,10,Bintulu,MY,3.1667,113.0333,77.41,100,20,0.00,few clouds
18,18,Hambantota,LK,6.1241,81.1185,79.14,94,75,4.61,broken clouds
20,20,Sambava,MG,-14.2667,50.1667,79.61,84,31,14.34,light rain
21,21,Castro,BR,-24.7911,-50.0119,75.07,65,95,5.37,overcast clouds
23,23,Lubango,AO,-14.9172,13.4925,82.26,28,40,4.61,scattered clouds


In [99]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

WeatherPy_Database.csv    196
City                      196
Country                   192
Lat                       196
Lng                       196
Max Temp                  196
Humidity                  196
Cloudiness                196
Wind Speed                196
Current Description       196
dtype: int64

In [100]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

WeatherPy_Database.csv    192
City                      192
Country                   192
Lat                       192
Lng                       192
Max Temp                  192
Humidity                  192
Cloudiness                192
Wind Speed                192
Current Description       192
dtype: int64

In [101]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Atuona,PF,78.84,light rain,-9.8000,-139.0333,
4,Rikitea,PF,78.89,clear sky,-23.1203,-134.9692,
6,Dingle,PH,76.10,overcast clouds,10.9995,122.6711,
7,Tabou,CI,86.07,overcast clouds,4.4230,-7.3528,
9,Waingapu,ID,78.82,overcast clouds,-9.6567,120.2641,
10,Bintulu,MY,77.41,few clouds,3.1667,113.0333,
18,Hambantota,LK,79.14,broken clouds,6.1241,81.1185,
20,Sambava,MG,79.61,light rain,-14.2667,50.1667,
21,Castro,BR,75.07,overcast clouds,-24.7911,-50.0119,
23,Lubango,AO,82.26,scattered clouds,-14.9172,13.4925,


In [102]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [103]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df = hotel_df.replace("", nan_value)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Atuona,PF,78.84,light rain,-9.8000,-139.0333,Villa Enata
4,Rikitea,PF,78.89,clear sky,-23.1203,-134.9692,People ThankYou
6,Dingle,PH,76.10,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
7,Tabou,CI,86.07,overcast clouds,4.4230,-7.3528,hotêl doufoulougou
9,Waingapu,ID,78.82,overcast clouds,-9.6567,120.2641,Pondok Wisata Elim
...,...,...,...,...,...,...,...
641,Itaituba,BR,82.56,broken clouds,-4.2761,-55.9836,Hotel Campos
644,Kayankulam,IN,77.20,light rain,9.1833,76.5000,Hotel Indraprastha
646,Uthal,PK,85.86,broken clouds,25.8072,66.6219,Al-Mustafa Hotel
648,Tual,ID,84.11,broken clouds,-5.6667,132.7500,Grand Vilia Hotel


In [104]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [105]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [109]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))